In [57]:
# Based on: https://medium.com/@ilaslanduzgun/next-word-prediction-using-lstm-with-tensorflow-e2a8f63b613c

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [58]:
text = open('mobydick-gutenberg.txt', 'r', encoding='utf-8').read()

In [59]:
tokenizer = Tokenizer()
#fit
tokenizer.fit_on_texts([text])
#assign length of word index
total_words = len(tokenizer.word_index) + 1

In [84]:
#declare ngrams
input_sequences = []
#split the sentence from '\n'
for line in text.split('\n')[0:1000]:
    #get tokens
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [100]:
input_sequences[4]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0, 10189,   265,   620,  1601,     2,   284],
      dtype=int32)

In [85]:
sentence_token = input_sequences[19] #
sentence = []
for token in sentence_token:
    sentence.append(list((tokenizer.word_index).keys())[list((tokenizer.word_index).values()).index(token)])
print(sentence)

['this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united']


In [86]:
#maximum sentence length
max_sequence_len = max([len(seq) for seq in input_sequences])
# input sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [101]:
print(max_sequence_len)

17


In [87]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [88]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 16, 100)           1959200   
                                                                 
 lstm_6 (LSTM)               (None, 150)               150600    
                                                                 
 dense_6 (Dense)             (None, 19592)             2958392   
                                                                 
Total params: 5068192 (19.33 MB)
Trainable params: 5068192 (19.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [89]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#fit the model
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
183/183 [==============================] - 4s 20ms/step - loss: 7.7865 - accuracy: 0.0752
Epoch 2/50
183/183 [==============================] - 4s 20ms/step - loss: 6.4454 - accuracy: 0.0764
Epoch 3/50
183/183 [==============================] - 4s 20ms/step - loss: 6.2300 - accuracy: 0.0768
Epoch 4/50
183/183 [==============================] - 4s 20ms/step - loss: 6.0879 - accuracy: 0.0764
Epoch 5/50
183/183 [==============================] - 3s 19ms/step - loss: 5.9847 - accuracy: 0.0797
Epoch 6/50
183/183 [==============================] - 4s 20ms/step - loss: 5.8927 - accuracy: 0.0884
Epoch 7/50
183/183 [==============================] - 4s 20ms/step - loss: 5.7853 - accuracy: 0.0934
Epoch 8/50
183/183 [==============================] - 4s 20ms/step - loss: 5.6650 - accuracy: 0.1022
Epoch 9/50
183/183 [==============================] - 4s 20ms/step - loss: 5.5276 - accuracy: 0.1263
Epoch 10/50
183/183 [==============================] - 4s 20ms/step - loss: 5.3956 - accura

In [92]:
#determine a text
seed_text = "The whale was "
# predict word number
next_words = 7

for _ in range(next_words):
    #convert to token
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    #path sequences
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    #model prediction
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    # get predict words
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

1/1 [==============================] - 0s 8ms/step


In [93]:
seed_text

'The whale was  directly over him and probably killed him'

In [94]:
#determine a text
seed_text = "The whale was trying hard to "
# predict word number
next_words = 7

for _ in range(next_words):
    #convert to token
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    #path sequences
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    #model prediction
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    # get predict words
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

1/1 [==============================] - 0s 9ms/step


In [95]:
seed_text

'The whale was trying hard to  the whale is doubtless the tormenting mild'